In [1]:

import scipy.io
# Load the .mat file
file_path = '../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
data.shape

(50, 1000, 832)

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

from iTransformer import iTransformer2D

class TimeSeriesDataset(Dataset):
    def __init__(self, data, lookback_len, forecast_len):
        self.data = data
        self.lookback_len = lookback_len
        self.forecast_len = forecast_len

    def __len__(self):
        return self.data.shape[1] - self.lookback_len - self.forecast_len

    def __getitem__(self, idx):
        x = self.data[:, idx:idx + self.lookback_len, :]
        y = self.data[:, idx + self.lookback_len:idx + self.lookback_len + self.forecast_len, :]
        return x[0], y[0]  # Ensure correct shape for batch dimension


In [3]:
num_samples = 50
num_timesteps = 1000
num_features = 832


In [4]:
# Define model parameters
lookback_len = 10  # Input timesteps
forecast_len = 1   # Prediction horizon


In [5]:
data.shape

(50, 1000, 832)

In [6]:

# Initialize the dataset and dataloader
dataset = TimeSeriesDataset(data, lookback_len, forecast_len)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)



In [7]:
# Define the iTransformer2D model
model = iTransformer2D(
    num_variates=num_features,
    lookback_len=lookback_len,
    dim=832,
    depth=6,
    heads=8,
    dim_head=64,
    pred_length=forecast_len,
    num_time_tokens=2,
    use_reversible_instance_norm=True
)


Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [8]:
# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [9]:

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs = torch.tensor(inputs, dtype=torch.float32).clone().detach().requires_grad_(True)
        targets = torch.tensor(targets, dtype=torch.float32).clone().detach().requires_grad_(True)

        # Ensure inputs and targets have the correct shape
        print(inputs.shape)  # Debug print to check shape

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


/tmp/ipykernel_44558/2962362001.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).clone().detach().requires_grad_(True)
/tmp/ipykernel_44558/2962362001.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.float32).clone().detach().requires_grad_(True)


torch.Size([4, 10, 832])


AttributeError: 'dict' object has no attribute 'size'

In [ ]:

# Print model summary to verify the configuration (if supported by the class)
try:
    model.summary()
except AttributeError:
    print("Model summary method not available. Check model parameters manually.")
